# 다형성과 유형 클래스

## 매개변수 다형성

다형유형의 정의를 **특정 형식의 유형을 임의의 유형을 이용해서 생성할 수 있도록 하는 기법**으로 이해할 수 있다. 
이전에 살펴보았던 리스트, 튜플, 함수 유형 모두 임의의 유형을 사용하여 동일한 형식의 새로운 유형을 생성한다.

* 리스트 유형: `[Int]`, `[Float]`, `[String]`, `[Bool]`, 등등.

* 튜플 유형: `(Int, String)`, `(Float, Bool)`, `(String, Char)`, 등등.

* 함수 유형: `Int -> Float`, `(Int, Int) -> Int`, `[Bool] -> Int`, 등등.

다형유형은 하스켈이 지원하는 다형성polymorphism의 핵심이며, 
앞서 설명한 이유로 해서 **매개변수 다형성**parametric polymorphism이라고도 불린다. 

`[a]`, `(a, b)`, `a -> b` 등의 경우처럼 유형 변수type variable를 
임의의 유형을 인자로 받을 수 있는 일종의 매개변수로 간주한다는 의미에서 매개변수 다형성이라 부른다고 이해할 수 있다. 

### 다형 유형의 엄밀한 의미

`t :: [a]`, `t :: (a, b)`, `t :: a -> b`의 엄밀한 의미는 다음과 같다.

* `t :: forall a. [a]`
    * 임의의 유형 `A`에 대해 `t A :: [A]`가 성립한다. 
        즉, `t A`는 `A` 유형의 값들로 이루어진 리스트이다. 
    * 예를 들어, `t Int`는 정수들의 리스트이어야 한다. 
* `t :: forall a b. (a, b)`
    * 임의의 유형 `A, B`에 대해 `t A B :: (A, B)`가 성립한다. 
        즉, `t A B`는 `A` 유형의 값과 `B` 유형의 값으로 이루어진 길이가 2인 튜플이다. 
    * 예를 들어, `t Int Int`는 두 개의 정수로 이루어진 순선쌍이어야 한다. 
* `t :: forall a b. a -> b`
    * 임의의 유형 `A, B`에 대해 `t A B :: A -> B`가 성립한다. 
        즉, `t A B`는 `A` 유형에서 `B` 유형으로 가는 함수이다. 
    * 예를 들어, `t (Int,Int) Int`는 길이가 2인 정수들의 순선쌍에 적용하면 새로운 정수를 계산해야 한다. 

### 유형 추론

다형유형의 함수를 사용할 때 앞서 설명한 대로 사용되는 특정 유형을 지정하는 일은 매우 귀찮은 일이다.
하지만 일반적으로 그럴 필요가 없다. 
하스켈이 사용되는 값의 유형을 기본적으로 스스로 유추할 수 있기 때문이며, 
이를 **유형 추론**type inference이라 부른다. 

**주의:** 유형 추론이 경우에 따라 불가능 할 수 있다. 
예를 들어, 공 리스트 `[]`와 정수 `1, 2, 3` 등의 유형이 명확하지 않을 수 있다. 
공 리스트는 임의의 유형의 리스트일 수 있으며, `1, 2, 3` 등의 숫자는 `Int`, `Integer` 등이 될 수 있기 때문이다.
하스켈은 유형에 매우 엄격하기 때문에 `Int`를 기대하는 함수에 `Integer`를 적용하면 오류가 발생한다. 

#### 예제

리스트에 포함된 항목들의 순서를 바꾸는 `reverse` 함수를 살펴보자. 
`reverse`는 임의의 유형의 리스트에 대해 작동하며, 유형은 아래와 같다. 

In [1]:
:t reverse

reverse :: forall a. [a] -> [a]

`reverse` 함수를 리스트 `[True, True, False, False]`와 함께 적용하면 아래와 같이 작동한다. 

In [2]:
reverse [True, False, False]

[False,False,True]

엄밀히 따지면 `[True, False, False]`의 유형 `[Bool]`에 사용된 `a`를 `reverse`가 알아야 한다.
하지만 `[True, False, False]`를 보고서 `[a]`에 사용된 `a`가 `Bool`임을 추론할 수 있기에 
굳이 `a`에 대한 정보를 입력하지 않아도 되는 것이다. 

**주의:** 경우에 따라 유형을 명시해야 하는 경우가 발생한다. 그럴 경우 아래와 같이 하면 된다.

In [3]:
reverse [True, False, False] :: [Bool]

[False,False,True]

## 유형 클래스

다형성과 관련해서 하스켈이 제공하는 기능이 **중복정의**이다. 
예를 들어, 덧셈 연산자 `+`는 정수들의 덧셈뿐만 아니라 실수들의 덧셈도 계산한다. 

In [4]:
2 + 3

5

In [5]:
2.0 + 3.0

5.0

그런데 중복 정의는 임의의 유형에 대해서 작동하지는 않는다. 
앞서 살펴보았던 `reverse` 함수의 경우 임의의 유형의 리스트에 대해 작동하는 반면에 
덧셈 연산자는 정수, 부동소수점 등 특정 유형들에 대해서만 작동한다. 
예를 들어, `True + False`, `[1, 2] + [0, 3]`의 값은 아예 정의되어 있지 않다.

### 오버로딩과 애드혹 다형성

이렇듯 몇몇 특정 유형들에 대해서 제한되지만 동일한 이름(또는 기호)의 함수를 사용하는 것을 
**오버로딩**(중복정의)이라 부른다.
오버로딩은 허용되는 유형을 제한한다는 의미에서 매개변수 다형성과 다르며, 
일명 **애드혹 다형성**ad hoc polymorphism이라고도 한다.

애드혹ad hoc을 "특별히, 임시방편의" 등과 같은 사전적 의미 보다는 
**특정 유형들로 제한되어 사용된다**는 의미로 이해해야 하며, 
그런 특정 유형들의 모음을 **유형 클래스**type class라 부른다. 

예를 들어, 덧셈 연산자 `+`가 사용될 수 있는 유형들의 모음은 `Num` 이라고 불리는 유형 클래스이다. 
실제로 `+`의 유형을 확인해보자.

**주의:** 덧셈 `+`, 뺄셈 `-`, 곱셈 `*`, 나눗셈 `/` 등의 중위infix 연산자들의 유형을 확인할 때는 
괄호로 감싸서 전위prefix 연산자로 만들어야 한다. 

In [9]:
:t (+)

(+) :: forall a. Num a => a -> a -> a

위 유형을 설명하면 다음과 같다.

* `a -> a -> a`: 
    * 임의의 유형 `a`에 대해,
    * `a` 유형의 값 두 개에 적용하면 `a` 유형의 값을 계산한다.
* `Num a => a -> a -> a`
    * 단, `a`는 `Num` 클래스의 인스턴스이어야 한다.
    * 즉, 덧셈은 `Num` 클래스에 속하는 유형에 대해서만 작동한다.

뺄셈과 나눗셈도 동일한 유형을 갖는다. 

In [11]:
:t (-)

(-) :: forall a. Num a => a -> a -> a

In [12]:
:t (*)

(*) :: forall a. Num a => a -> a -> a

반면에 나눗셈의 유형은 조금 다르다.

In [14]:
:t (/)

(/) :: forall a. Fractional a => a -> a -> a

* `Fractional a => a -> a -> a`
    * 단, `a`는 `Fractional` 클래스의 인스턴스이어야 한다.
    * 즉, 나눗셈은 `Fractional` 클래스에 속하는 유형에 대해서만 작동한다.

사칙연산 이외에 일반적으로 많이 사용되는 연산자들 모두 특정 유형 클래스의 인스턴스들을 대상으로 중복 정의되어 있다. 

#### 예제: `negate`

양수를 음수로, 음수를 양수로 기호 반전시키는 반전함수 `negate`도 `Num` 클래스의 인스턴스에 대해서만 정의된다.

In [15]:
:t negate

negate :: forall a. Num a => a -> a

`negate` 함수의 기호는 빼기 기호 `-`와 동일하다. 

In [22]:
-3 + 3

0

In [24]:
3 + (-3)

0

빼기와 `negate`의 기호는 동일하지만 사용되는 위치의 역할에 따라 구분된다.
이 부분은 파싱parsing의 기능에 의존한다. 
파싱에 대한 논의는 여기서는 하지 않는다.
다만 사용에 주의해야 한다는 점은 기억해야 한다.

예를 들어 아래와 같이 사용하면 파싱 오류가 발생한다. 
이유는 뺄셈으로 파싱할지, 아니면 반전 함수로 파싱할지 명확하지 않기 때문이다. 

In [25]:
3 + -3

: 

#### 예제: `abs`

절댓값을 계산하는 `abs`도 `Num` 클래스의 인스턴스에 대해서만 작동한다.

In [26]:
:t abs

abs :: forall a. Num a => a -> a

#### 예제: 숫자

1, 2, 3, ... 등의 정수, 1.0, 2.3, 3.8 등의 부동소수점도 여러 유형에 대해 중복 정의되어 있다. 
즉, 우리에게 보이는 숫자들이 특정 유형에서 공통으로 사용되는 **상수constant 기호**라는 의미이다.

정수는 `Num` 클래스의 인스턴스 유형에서 공통으로 사용되는 상수 기호이다. 

In [30]:
:t 1

1 :: forall p. Num p => p

부동소수점은 `Fractional` 클래스의 인스턴스 유형에서 공통으로 사용되는 상수 기호이다. 

In [31]:
:t 3.1

3.1 :: forall p. Fractional p => p

In [32]:
:t elem

elem :: forall (t :: * -> *) a. (Foldable t, Eq a) => a -> t a -> Bool

## 유형 클래스와 객체 지향 프로그래밍